
### Entrada
- x1: Local da Coleta em km
- x2: Nivel da Mare em m
- x3: Umidade do Ar (%)
- x4: Índice Pluvimétrico (mm)¹
- x5: Temperatura (°C)

### Saida
- y1: pH
- y2: Condutividade Elétrica (mS.cmˉ¹)
- y3: Oxigênio Dissolvido (mg.Lˉ¹)
- y4: Sólidos Totais Dissolvidos (ppm)


In [1]:
!pip install --upgrade pip
!pip install tensorflow 
!pip install openpyxl

In [2]:
import os

os.chdir("./content")

lm_dir = "tf-levenberg-marquardt"
if not os.path.exists(lm_dir):
  !git clone https://github.com/fabiodimarco/$lm_dir

os.chdir(lm_dir)

In [3]:
import numpy as np
import pandas as pd

def create_dataframe(output, label):
    df = pd.read_excel("../../../data.xlsx")
    x1, x2, x3, x4, x5  = df['x1'], df['x2'], df['x3'], df['x4'], df['x5']

    input = np.vstack([x1, x2, x3, x4, x5 ]).T
    output = np.array(df[output])
    return output, input

output, input = create_dataframe(output="y3", label="oxigenio")

/tmp/ipykernel_3075419/160627683.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [4]:
import tensorflow as tf
import numpy as np
from keras import regularizers
from keras import initializers
import levenberg_marquardt as lm

# layers, neurons
class ShuffleArchitecture:
    def __init__(self, input_size, hidden_sizes, output_size, act_h, act_o, param_reg):
        self.input_size = input_size
        self.hidden_sizes = hidden_sizes
        self.output_size = output_size
        self.act_h = act_h
        self.act_o = act_o
        self.regularizer = regularizers.L2(param_reg)
        self.initializer = initializers.RandomUniform(minval=-0.5, maxval=0.5, seed=np.random.randint(1, 10000))

    def set_architecture(self):
        self.model = tf.keras.Sequential()
        self.model.add(tf.keras.layers.Dense(self.hidden_sizes[0],
                        input_shape=(self.input_size,),
                        activation=self.act_h,
                        kernel_regularizer=self.regularizer,
                        kernel_initializer=self.initializer,                        
                        ))  # input layer
        
        for size in self.hidden_sizes[1:]:  # hidden layers
            self.model.add(tf.keras.layers.Dense(size,
                            activation=self.act_h,
                            kernel_regularizer=self.regularizer,
                            kernel_initializer=self.initializer,  
                        ))

        self.model.add(tf.keras.layers.Dense(self.output_size,
                        activation=self.act_o,
                        kernel_regularizer=self.regularizer,
                        kernel_initializer=self.initializer,  
                        ))  # output layer

    def create_model(self, _learning_rate):
        self.model.compile(
            optimizer=tf.keras.optimizers.Adam(learning_rate=_learning_rate),
            loss=tf.keras.losses.MeanSquaredError())

        self.lm_model = lm.ModelWrapper(
            tf.keras.models.clone_model(self.model))

        self.lm_model.compile(
            optimizer=tf.keras.optimizers.SGD(learning_rate=_learning_rate),
            loss=lm.MeanSquaredError())
        return(self.lm_model)

2024-03-26 02:04:58.549926: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-26 02:04:58.596479: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-26 02:04:58.596523: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-26 02:04:58.597900: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-26 02:04:58.606212: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-26 02:04:58.607462: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [5]:
import tensorflow as tf
import matplotlib.pyplot as plt
import time
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping
from sklearn.metrics import r2_score, mean_squared_error


class TrainWithSmallDataset:
    def __init__(self, batch_size=1000):
        self.batch_size = batch_size
        self.betters = []
    
    def create_dataset(self, input, output):
      input = tf.expand_dims(tf.cast(input, tf.float32), axis=-1)
      output = tf.expand_dims(tf.cast(output, tf.float32), axis=-1)

      dataset = tf.data.Dataset.from_tensor_slices((input, output))
      dataset = dataset.shuffle(len(input))
      dataset = dataset.batch(self.batch_size).cache()
      dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)
      return (dataset, input, output)

    def split_dataset(self, input, output):
      input_train, input_vt, output_train, output_vt = train_test_split(input, output, test_size=0.3, shuffle = True)
      input_val, input_test, output_val, output_test = train_test_split(input_vt, output_vt, test_size=0.5, shuffle = True)

      self.train_dataset, self.train_input, self.train_output = self.create_dataset(input_train, output_train)
      self.val_dataset, self.val_input, self.val_output = self.create_dataset(input_val, output_val)
      self.test_dataset, self.test_input, self.test_output = self.create_dataset(input_test, output_test)
      self.vt_dataset, self.vt_input, self.vt_output = self.create_dataset(input_vt, output_vt)
      self.dataset, self.input, self.output = self.create_dataset(input, output)

      self._train = (input_train, output_train)
      self._vt = (input_vt, output_vt)
      self._val = (input_val, output_val)
      self._test = (input_test, output_test)
 
 
    
    def train_using_lm(self, train_dataset, epochs=1000):
      early_stopping_monitor = EarlyStopping(monitor='val_loss',
                                              patience=6,
                                              restore_best_weights=True)

      t2_start = time.perf_counter()
      self.results = self.lm_model.fit(train_dataset,
                                            epochs=epochs,
                                            validation_data=self.val_dataset,
                                            callbacks=[early_stopping_monitor],
                                            verbose=0)
      t2_stop = time.perf_counter()
      print("Elapsed time: ", t2_stop - t2_start)
      print ("Stopped at epoch: ", early_stopping_monitor.stopped_epoch)

    def get_metrics(self):
          self.test_prediction = self.lm_model.predict(self.test_input)

          pred = self.lm_model.predict(self.input).flatten()
          test_pred = self.test_prediction.flatten()
          val_pred = self.lm_model.predict(self.val_input).flatten()
          vt_pred = self.lm_model.predict(self.vt_input).flatten()

          r2 = r2_score(self.output, pred)
          r2_test = r2_score(self.test_output, test_pred)
          r2_val = r2_score(self.val_output, val_pred)
          r2_vt = r2_score(self.vt_output, vt_pred)

          mse = mean_squared_error(self.output, pred)
          mse_test = mean_squared_error(self.test_output, test_pred)
          mse_val = mean_squared_error(self.val_output, val_pred)
          mse_vt = mean_squared_error(self.vt_output, vt_pred)

          metrics = {
                          'r2': r2,
                          'r2_test': r2_test,
                          'r2_val': r2_val,
                          'r2_vt': r2_vt,
                          'mse': mse,
                          'mse_test': mse_test,
                          'mse_val': mse_val,
                          'mse_vt': mse_vt
                          }

          return metrics

In [6]:
from itertools import product
import pickle

class Tester:
  def __init__(self, input, output, run_times=500, dataset_run_times=10):
    self.run_times = run_times
    self.better_metrics = {}
    self.dataset_run_times = dataset_run_times
    self.input, self.output = input, output
  
  def setArchitecure(self, trainer, _hidden_sizes, _pg, _lr):
    shuffler = ShuffleArchitecture(input_size=5,
                                    hidden_sizes=_hidden_sizes,
                                    output_size=1,
                                    act_h='tanh',
                                    act_o='linear',
                                    param_reg=_pg)
    shuffler.set_architecture()    
    trainer.lm_model = shuffler.create_model(_lr)

  def Train(self, trainer, epochs=1000):
    trainer.train_using_lm(trainer.train_dataset, epochs=epochs)
    return(trainer.get_metrics(), trainer.lm_model)

  def SaveModelWeights(self, model, fileName):
    path = f"../models/{fileName}.keras"
    open(path,'w').close()
    model.save_weights(path) 

  def SaveDataset(self, trainer, fileName):
    path = f"../dataset/{fileName}.pkl" 
    with open(path, 'wb') as f:
      pickle.dump((trainer._train, trainer._vt, trainer._val, trainer._test), f)


  def LoopWeights(self, sort_by, boundarie, trainer, idx):
    better_model = 0
    save = False

    for i in range(self.run_times):
      print (f"+++++++++++ [{idx}] | {i + 1} ++++++++++++++++++")
      metrics, model = self.Train(trainer)
      if (metrics[sort_by] >= boundarie): # should be <= to descending metrics
        fileName = f"model_{idx}_{better_model}"
        self.SaveModelWeights(model, fileName)
        self.better_metrics[fileName] = metrics
        better_model += 1
        save = True
    
    return(save)

# Init
  def Loop(self, sort_by, boundarie, hidden_sizes, regularizers, learning_rate):
    trainer = TrainWithSmallDataset()

    for count, (hidden_size, reg, lr) in enumerate(product(hidden_sizes, regularizers, learning_rate), start=1):
      header =  f"Hidden Size={hidden_size}, regularizer={reg}, learning_rate={lr}"
      print(f"Testando combinacao{count}: {header}")
      self.setArchitecure(trainer, hidden_size, reg, lr)
      for j in range(self.dataset_run_times):
        trainer.split_dataset(self.input, self.output)
        if (self.LoopWeights(sort_by, boundarie, trainer, f"{count}_{j}") == True):
          self.SaveDataset(trainer, f"dataset_{count}_{j}")
          self.DisplayBetterResults(sort_by, header, f"{count}_{j}")
        self.better_metrics = {}

  def DisplayBetterResults(self, sort_by, header, dataset=0):
    df = pd.DataFrame.from_dict(self.better_metrics, orient='index')
    df = df.sort_values([sort_by])
    display(df)
    path = f'../results/metrics_{dataset}'
    df.to_excel(f"{path}.xlsx", index=True)
    print(f"DataFrame salvo em {path}")
    with open(f"{path}.txt", 'w') as arquivo:
      arquivo.write(header)

In [7]:
tester = Tester(input, output, run_times=25, dataset_run_times=10)

In [8]:
tester.Loop(sort_by='r2',
            boundarie=0.8,
            hidden_sizes = [[15,7]],
            regularizers=[ 0.1,0.02],
            learning_rate=[ 0.1,0.02])

Testando combinacao1: Hidden Size=[15, 7], regularizer=0.1, learning_rate=0.1
+++++++++++ [1_0] | 1 ++++++++++++++++++


Elapsed time:  1.362496262001514
Stopped at epoch:  10
1/1 [==============================] - 0s 43ms/step
+++++++++++ [1_0] | 2 ++++++++++++++++++
Elapsed time:  0.5605496809985198
Stopped at epoch:  9
1/1 [==============================] - 0s 25ms/step
+++++++++++ [1_0] | 3 ++++++++++++++++++
Elapsed time:  1.5543899779986532
Stopped at epoch:  20
1/1 [==============================] - 0s 22ms/step
+++++++++++ [1_0] | 4 ++++++++++++++++++
Elapsed time:  0.366274218999024
Stopped at epoch:  6
1/1 [==============================] - 0s 16ms/step
+++++++++++ [1_0] | 5 ++++++++++++++++++
Elapsed time:  0.4615833140014729
Stopped at epoch:  8
1/1 [==============================] - 0s 17ms/step
+++++++++++ [1_0] | 6 ++++++++++++++++++
Elapsed time:  0.5708395099991321
Stopped at epoch:  6
1/1 [==============================] - 0s 16ms/step
+++++++++++ [1_0] | 7 ++++++++++++++++++
Elapsed time:  0.4752515480013244
Stopped at epoch:  6
1/1 [==============================] - 0s 16ms/step
+++++

,r2,r2_test,r2_val,r2_vt,mse,mse_test,mse_val,mse_vt
model_2_2_0,0.812963,0.702781,-3.955398,0.435714,0.206995,0.281732,0.326406,0.302755
model_2_2_1,0.845918,0.823813,-4.683853,0.506832,0.170523,0.167006,0.374388,0.264598
model_2_2_4,0.845944,0.758753,-4.647088,0.448105,0.170494,0.228676,0.371967,0.296107
model_2_2_2,0.847858,0.818109,-4.656712,0.503064,0.168376,0.172413,0.372601,0.266619
model_2_2_3,0.853563,0.834502,-4.604112,0.521436,0.162063,0.156874,0.369136,0.256762
model_2_2_22,0.874551,0.496975,-3.899633,0.246439,0.138835,0.476813,0.322733,0.404306
model_2_2_20,0.874793,0.493078,-3.796759,0.248737,0.138568,0.480507,0.315957,0.403072
model_2_2_21,0.876211,0.508584,-3.888861,0.257921,0.136998,0.465809,0.322023,0.398145
model_2_2_5,0.882376,0.773998,-3.392007,0.534873,0.130176,0.214225,0.289296,0.249553
model_2_2_6,0.884203,0.777690,-3.546135,0.529422,0.128153,0.210725,0.299448,0.252477


DataFrame salvo em ../results/metrics_2_2
+++++++++++ [2_3] | 1 ++++++++++++++++++
Elapsed time:  0.26943725499950233
Stopped at epoch:  6
1/1 [==============================] - 0s 16ms/step
+++++++++++ [2_3] | 2 ++++++++++++++++++
Elapsed time:  0.25575344400022004
Stopped at epoch:  6
1/1 [==============================] - 0s 15ms/step
+++++++++++ [2_3] | 3 ++++++++++++++++++
Elapsed time:  0.3448685919993295
Stopped at epoch:  8
1/1 [==============================] - 0s 16ms/step
+++++++++++ [2_3] | 4 ++++++++++++++++++
Elapsed time:  0.3233760160001111
Stopped at epoch:  6
1/1 [==============================] - 0s 22ms/step
+++++++++++ [2_3] | 5 ++++++++++++++++++
Elapsed time:  0.29369747600139817
Stopped at epoch:  6
1/1 [==============================] - 0s 15ms/step
+++++++++++ [2_3] | 6 ++++++++++++++++++
Elapsed time:  0.3093984650004131
Stopped at epoch:  6
1/1 [==============================] - 0s 20ms/step
+++++++++++ [2_3] | 7 ++++++++++++++++++
Elapsed time:  0.428136215

,r2,r2_test,r2_val,r2_vt,mse,mse_test,mse_val,mse_vt
model_2_3_2,0.873943,0.310938,0.955382,0.829033,0.139508,0.314457,0.076663,0.202554
model_2_3_0,0.876680,0.871317,0.983815,0.962713,0.136479,0.058725,0.027808,0.044176
model_2_3_1,0.878767,0.858146,0.980969,0.958085,0.134169,0.064736,0.032699,0.049660
model_2_3_3,0.887388,0.461611,0.951066,0.856810,0.124628,0.245697,0.084079,0.169644


...

DataFrame salvo em ../results/metrics_4_7
+++++++++++ [4_8] | 1 ++++++++++++++++++
Elapsed time:  0.5078933740005596
Stopped at epoch:  9
1/1 [==============================] - 0s 30ms/step
+++++++++++ [4_8] | 2 ++++++++++++++++++
Elapsed time:  0.5196507720011141
Stopped at epoch:  10
1/1 [==============================] - 0s 15ms/step
+++++++++++ [4_8] | 3 ++++++++++++++++++
Elapsed time:  0.42417015899991384
Stopped at epoch:  6
1/1 [==============================] - 0s 22ms/step
+++++++++++ [4_8] | 4 ++++++++++++++++++
Elapsed time:  0.3780167019995133
Stopped at epoch:  6
1/1 [==============================] - 0s 16ms/step
+++++++++++ [4_8] | 5 ++++++++++++++++++
Elapsed time:  0.3669150650002848
Stopped at epoch:  6
1/1 [==============================] - 0s 15ms/step
+++++++++++ [4_8] | 6 ++++++++++++++++++
Elapsed time:  0.36103354900114937
Stopped at epoch:  6
1/1 [==============================] - 0s 21ms/step
+++++++++++ [4_8] | 7 ++++++++++++++++++
Elapsed time:  0.390461208

,r2,r2_test,r2_val,r2_vt,mse,mse_test,mse_val,mse_vt
model_4_8_24,0.808290,-0.115024,0.626521,0.496811,0.212166,0.564205,0.351069,0.463906
model_4_8_23,0.808392,-0.114103,0.626868,0.497246,0.212053,0.563739,0.350743,0.463505
model_4_8_22,0.808783,-0.110385,0.628021,0.498879,0.211620,0.561858,0.349660,0.462000
model_4_8_21,0.808810,-0.110091,0.628077,0.498991,0.211591,0.561709,0.349607,0.461896
model_4_8_20,0.810562,-0.094775,0.633698,0.506139,0.209652,0.553959,0.344323,0.455307
model_4_8_14,0.811376,-0.078067,0.634465,0.511362,0.208751,0.545505,0.343602,0.450492
model_4_8_19,0.811815,-0.084051,0.638647,0.511629,0.208265,0.548533,0.339671,0.450245
model_4_8_18,0.812092,-0.080741,0.639391,0.512948,0.207958,0.546858,0.338972,0.449029
model_4_8_17,0.812336,-0.077547,0.640038,0.514186,0.207689,0.545241,0.338364,0.447887
model_4_8_13,0.812575,-0.068545,0.639443,0.516517,0.207424,0.540687,0.338923,0.445739


DataFrame salvo em ../results/metrics_4_8
+++++++++++ [4_9] | 1 ++++++++++++++++++
Elapsed time:  0.4680218510002305
Stopped at epoch:  8
1/1 [==============================] - 0s 15ms/step
+++++++++++ [4_9] | 2 ++++++++++++++++++
Elapsed time:  0.3940288979993056
Stopped at epoch:  7
1/1 [==============================] - 0s 21ms/step
+++++++++++ [4_9] | 3 ++++++++++++++++++
Elapsed time:  0.5145425439986866
Stopped at epoch:  8
1/1 [==============================] - 0s 16ms/step
+++++++++++ [4_9] | 4 ++++++++++++++++++
Elapsed time:  0.44673720199898526
Stopped at epoch:  6
1/1 [==============================] - 0s 16ms/step
+++++++++++ [4_9] | 5 ++++++++++++++++++
Elapsed time:  0.5583234770001582
Stopped at epoch:  6
1/1 [==============================] - 0s 18ms/step
+++++++++++ [4_9] | 6 ++++++++++++++++++
Elapsed time:  0.622008308999284
Stopped at epoch:  9
1/1 [==============================] - 0s 19ms/step
+++++++++++ [4_9] | 7 ++++++++++++++++++
Elapsed time:  0.363187724000

,r2,r2_test,r2_val,r2_vt,mse,mse_test,mse_val,mse_vt
model_4_9_2,0.806153,0.773035,0.812299,0.797584,0.214532,0.416259,0.309491,0.366016
model_4_9_4,0.807661,0.758951,0.796190,0.783109,0.212863,0.442089,0.336053,0.392190
model_4_9_5,0.809027,0.746949,0.804191,0.780098,0.211351,0.464100,0.322860,0.397635
model_4_9_3,0.813652,0.775579,0.835450,0.808885,0.206233,0.411594,0.271319,0.345582
model_4_9_1,0.816593,0.786246,0.853429,0.822328,0.202977,0.392029,0.241673,0.321273
model_4_9_0,0.822056,0.941408,0.936578,0.941323,0.196932,0.107460,0.104573,0.106102


DataFrame salvo em ../results/metrics_4_9
